# Setup

In [10]:
CASSANDRA_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
DOCKER_DNS = ["10.15.20.1"]

CASSANDRA_CLUSTER_NAME = "cassandra-cluster.mavasbel.vpn.itam.mx"
CASSANDRA_TOTAL_NODES = 3

CASSANDRA_NODE_IPS = ["10.15.20.2"] * CASSANDRA_TOTAL_NODES
CASSANDRA_NODE_NAMES = [f"cassandra-node-{i+1}" for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_HOSTNAMES = [
    f"{CASSANDRA_NODE_NAMES[i]}.mavasbel.vpn.itam.mx"
    for i in range(CASSANDRA_TOTAL_NODES)
]
CASSANDRA_NODE_GOSSIP_PORTS = [7000 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_RPC_PORTS = [9040 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_SSL_GOSSIP_PORTS = [7500 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_JMX_PORTS = [7200 + (i + 1) for i in range(0, CASSANDRA_TOTAL_NODES)]

CASSANDRA_CA_CERT_PASSWORD = "cassandra_cluster_ca_cert_passowrd"
CASSANDRA_NODE_CERT_PASSWORD = "cassandra_cluster_cert_passowrd"
CASSANDRA_INIT_USER = "cassandra"
CASSANDRA_INIT_PASSWORD = "cassandra"

CASSANDRA_WORKDIR = "/var/lib/cassandra"

In [11]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")
CASSANDRA_LOCALHOST_CLUSTER_CA_CERTDIR = os.path.join(LOCALHOST_WORKDIR, "cluster_certs")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)

# Stop cassandra.docker-compose.yml

In [12]:
!docker compose -f cassandra-cluster.docker-compose.yml down -v

 Container cassandra-node-3  Stopping
 Container cassandra-node-3  Stopped
 Container cassandra-node-3  Removing
 Container cassandra-node-3  Removed
 Container cassandra-node-2  Stopping
 Container cassandra-node-2  Stopped
 Container cassandra-node-2  Removing
 Container cassandra-node-2  Removed
 Container cassandra-node-1  Stopping
 Container cassandra-node-1  Stopped
 Container cassandra-node-1  Removing
 Container cassandra-node-1  Removed
 Network cassandra-cluster_cassandra-cluster  Removing
 Network cassandra-cluster_cassandra-cluster  Removed


In [13]:
import shutil

if CASSANDRA_START_FROM_SCRATCH:
    shutil.rmtree(DOCKER_MOUNTDIR, ignore_errors=True)
    shutil.rmtree(CASSANDRA_LOCALHOST_CLUSTER_CA_CERTDIR, ignore_errors=True)
    Path(DOCKER_MOUNTDIR).mkdir(parents=True, exist_ok=True)

# Cluster keystore generation

### Create cluster CA certificate

In [14]:
if not os.path.exists(CASSANDRA_LOCALHOST_CLUSTER_CA_CERTDIR):
    Path(CASSANDRA_LOCALHOST_CLUSTER_CA_CERTDIR).mkdir(
        parents=True, exist_ok=True
    )
    ca_cert_gen_command = " && ".join(
        [
            # 1. Generate Private Key & Keystore
            f"keytool -genkeypair -alias ca-{CASSANDRA_CLUSTER_NAME} -keyalg RSA -keysize 4096 -ext bc:c -validity 3650 "
            f"-keystore /certs/ca.keystore -storepass {CASSANDRA_CA_CERT_PASSWORD} -keypass {CASSANDRA_CA_CERT_PASSWORD} "
            f"-dname 'CN=ca-{CASSANDRA_CLUSTER_NAME}, O=Uni'",
            # 2. Export CA cert
            f"keytool -export -alias ca-{CASSANDRA_CLUSTER_NAME} -file /certs/ca.crt "
            f"-keystore /certs/ca.keystore -storepass {CASSANDRA_CA_CERT_PASSWORD}",
            # 3. Save private key
            f"openssl pkcs12 -in /certs/ca.keystore -nodes -nocerts -out /certs/ca.key -passin pass:{CASSANDRA_CA_CERT_PASSWORD}",
        ]
    )
    result = !docker run --rm --mount type=bind,source="{CASSANDRA_LOCALHOST_CLUSTER_CA_CERTDIR}",target=/certs cassandra:5.0 bash -c "{ca_cert_gen_command}"
    print("✅ Cassandra cluster CA certificate generated")

✅ Cassandra cluster CA certificate generated


### Create cassandra.yaml and cluster nodes certificates

In [15]:
import yaml

for i in range(0, CASSANDRA_TOTAL_NODES):

    if not os.path.exists(
        os.path.join(
            os.path.join(DOCKER_MOUNTDIR, CASSANDRA_NODE_NAMES[i]), "cassandra.yaml"
        )
    ):
        with open(
            os.path.join(LOCALHOST_WORKDIR, "cassandra.yaml"), "r"
        ) as cassandra_config_source_file:
            cassandra_config = yaml.load(
                cassandra_config_source_file, Loader=yaml.FullLoader
            )
            cassandra_config["cluster_name"] = CASSANDRA_CLUSTER_NAME
            cassandra_config["native_transport_port"] = CASSANDRA_NODE_RPC_PORTS[i]
            cassandra_config["authenticator"] = "PasswordAuthenticator"
            cassandra_config["authorizer"] = "CassandraAuthorizer"
            cassandra_config["server_encryption_options"] = (
                cassandra_config["server_encryption_options"] or {}
            )
            cassandra_config["server_encryption_options"].update(
                {
                    "internode_encryption": "all",
                    "enabled": True,
                    "keystore": "/etc/cassandra/certs/keystore.jks",
                    "keystore_password": CASSANDRA_NODE_CERT_PASSWORD,
                    "truststore": "/etc/cassandra/certs/truststore.jks",
                    "truststore_password": CASSANDRA_NODE_CERT_PASSWORD,
                    "require_client_auth": True,
                }
            )
            cassandra_config["client_encryption_options"] = (
                cassandra_config["client_encryption_options"] or {}
            )
            cassandra_config["client_encryption_options"].update(
                {
                    "enabled": False,
                }
            )
            Path(os.path.join(DOCKER_MOUNTDIR, CASSANDRA_NODE_NAMES[i])).mkdir(
                parents=True, exist_ok=True
            )
            with open(
                os.path.join(
                    os.path.join(DOCKER_MOUNTDIR, CASSANDRA_NODE_NAMES[i]),
                    "cassandra.yaml",
                ),
                "w",
            ) as node_cassandra_yaml_file:
                yaml.dump(
                    cassandra_config,
                    node_cassandra_yaml_file,
                    default_flow_style=False,
                    sort_keys=False,
                )
                
    if not os.path.exists(
        os.path.join(
            os.path.join(DOCKER_MOUNTDIR, CASSANDRA_NODE_NAMES[i]), "cassandra-rackdc.properties"
        )
    ):
        Path(os.path.join(DOCKER_MOUNTDIR, CASSANDRA_NODE_NAMES[i])).mkdir(
                parents=True, exist_ok=True
            )
        shutil.copyfile(
            os.path.join(LOCALHOST_WORKDIR, "cassandra-rackdc.properties"),
            os.path.join(
                os.path.join(DOCKER_MOUNTDIR, CASSANDRA_NODE_NAMES[i]),
                "cassandra-rackdc.properties",
            ),
        )

    node_certdir = os.path.join(DOCKER_MOUNTDIR, CASSANDRA_NODE_NAMES[i], "certs")
    if not os.path.exists(node_certdir):
        Path(node_certdir).mkdir(parents=True, exist_ok=True)
        node_cmd = " && ".join(
            [
                # 1. Generate Private Key & Keystore
                f"keytool -genkeypair -alias {CASSANDRA_NODE_NAMES[i]} -keyalg RSA -keysize 4096 -validity 365 "
                f"-keystore /certs/keystore.jks -storepass {CASSANDRA_NODE_CERT_PASSWORD} -keypass {CASSANDRA_NODE_CERT_PASSWORD} "
                f"-dname 'CN={CASSANDRA_NODE_NAMES[i]}, OU=Lab, O=Uni, C=US'",
                # 2. Generate CSR
                f"keytool -certreq -alias {CASSANDRA_NODE_NAMES[i]} -file /certs/{CASSANDRA_NODE_NAMES[i]}.csr "
                f"-keystore /certs/keystore.jks -storepass {CASSANDRA_NODE_CERT_PASSWORD}",
                # 3. Sign the CSR with the CA Master (generating the .crt)
                f"keytool -gencert -alias ca-{CASSANDRA_CLUSTER_NAME} -infile /certs/{CASSANDRA_NODE_NAMES[i]}.csr -outfile /certs/{CASSANDRA_NODE_NAMES[i]}.crt "
                f"-keystore /cacerts/ca.keystore -storepass {CASSANDRA_CA_CERT_PASSWORD} -validity 365",
                # 4. Import CA into Keystore (to complete the chain)
                f"keytool -import -alias ca-{CASSANDRA_CLUSTER_NAME} -file /cacerts/ca.crt "
                f"-keystore /certs/keystore.jks -storepass {CASSANDRA_NODE_CERT_PASSWORD} -noprompt",
                # 5. Import the Signed Certificate back into Keystore
                f"keytool -import -alias {CASSANDRA_NODE_NAMES[i]} -file /certs/{CASSANDRA_NODE_NAMES[i]}.crt "
                f"-keystore /certs/keystore.jks -storepass {CASSANDRA_NODE_CERT_PASSWORD}",
                # 6. Create the Truststore (contains only the CA cert)
                f"keytool -import -alias ca-{CASSANDRA_CLUSTER_NAME} -file /cacerts/ca.crt "
                f"-keystore /certs/truststore.jks -storepass {CASSANDRA_NODE_CERT_PASSWORD} -noprompt",
                # 7. Save Private Key as independent file
                f"openssl pkcs12 -in /certs/keystore.jks -nodes -nocerts -out /certs/{CASSANDRA_NODE_NAMES[i]}.key -passin pass:{CASSANDRA_NODE_CERT_PASSWORD}",
            ]
        )
        !docker run --rm --mount type=bind,source="{node_certdir}",target=/certs --mount type=bind,source="{CASSANDRA_LOCALHOST_CLUSTER_CA_CERTDIR}",target=/cacerts,readonly --entrypoint bash cassandra:5.0 -c "{node_cmd}"

Generating 4,096 bit RSA key pair and self-signed certificate (SHA384withRSA) with a validity of 365 days
	for: CN=cassandra-node-1, OU=Lab, O=Uni, C=US
Certificate was added to keystore
Certificate reply was installed in keystore
Certificate was added to keystore
Generating 4,096 bit RSA key pair and self-signed certificate (SHA384withRSA) with a validity of 365 days
	for: CN=cassandra-node-2, OU=Lab, O=Uni, C=US
Certificate was added to keystore
Certificate reply was installed in keystore
Certificate was added to keystore
Generating 4,096 bit RSA key pair and self-signed certificate (SHA384withRSA) with a validity of 365 days
	for: CN=cassandra-node-3, OU=Lab, O=Uni, C=US
Certificate was added to keystore
Certificate reply was installed in keystore
Certificate was added to keystore


# Start cassandra-cluster.docker-compose.yml

In [16]:
# docker exec cassandra-node-1 env JVM_OPTS="" nodetool -u cassandra -pw cassandra status
# docker exec -it cassandra-node-1 env CQLSH_PORT=9041 cqlsh -u cassandra -p cassandra
# !docker exec cassandra-node-1 env JVM_OPTS="" nodetool -u cassandra -pw cassandra describecluster

In [17]:
import os
import yaml
from IPython.display import Markdown, display

node_cpus = "2.0"
node_memory = "2G"
node_start_heap = "1G"
node_max_heap = "2G"
jmx_pass_path = os.path.join(DOCKER_MOUNTDIR, "jmxremote.password")
with open(jmx_pass_path, "w") as f:
    f.write(f"{CASSANDRA_INIT_USER} {CASSANDRA_INIT_USER}")

cassandra_compose_dict = {
    "name": "cassandra-cluster",
    "services": {},
    "networks": {"cassandra-cluster": {"driver": "bridge"}},
}

for i in range(0, CASSANDRA_TOTAL_NODES):

    jvm_opts = " ".join(
        [
            f"-Xms{node_start_heap}",
            f"-Xmx{node_max_heap}",
            f"-Dcassandra.native_transport_port={CASSANDRA_NODE_RPC_PORTS[i]}",
            f"-Dcassandra.storage_port={CASSANDRA_NODE_GOSSIP_PORTS[i]}",
            f"-Dcassandra.ssl_storage_port={CASSANDRA_NODE_SSL_GOSSIP_PORTS[i]}",
            f"-Dcassandra.broadcast_rpc_address={CASSANDRA_NODE_HOSTNAMES[i]}",
            f"-Dcassandra.broadcast_address={CASSANDRA_NODE_HOSTNAMES[i]}",
            "-Dcom.sun.management.jmxremote.authenticate=false",
            "-Dcom.sun.management.jmxremote.ssl=false",
            f"-Dcassandra.jmx.remote.port={CASSANDRA_NODE_JMX_PORTS[i]}",
        ]
    )

    cassandra_compose_dict["services"][CASSANDRA_NODE_NAMES[i]] = {
        "image": "cassandra:5.0",
        "container_name": CASSANDRA_NODE_NAMES[i],
        "environment": [
            "CASSANDRA_USER=cassandra",  # Forces ownership check
            f"MAX_HEAP_SIZE={node_max_heap}",
            f"HEAP_NEWSIZE={node_start_heap}",
            f"CASSANDRA_CLUSTER_NAME={CASSANDRA_CLUSTER_NAME}",
            f"CASSANDRA_SEEDS={','.join([f"{CASSANDRA_NODE_IPS[j]}:{CASSANDRA_NODE_GOSSIP_PORTS[j]}" for j in range(CASSANDRA_TOTAL_NODES)])}",
            "CASSANDRA_LISTEN_ADDRESS=auto",
            "CASSANDRA_RPC_ADDRESS=0.0.0.0",  # Listen on all interfaces
            "CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch",
            f"CASSANDRA_NATIVE_TRANSPORT_PORT={CASSANDRA_NODE_RPC_PORTS[i]}",
            f"CASSANDRA_STORAGE_PORT={CASSANDRA_NODE_GOSSIP_PORTS[i]}",
            f"CASSANDRA_SSL_STORAGE_PORT={CASSANDRA_NODE_SSL_GOSSIP_PORTS[i]}",
            f"JVM_OPTS={jvm_opts}",
            f"CASSANDRA_BROADCAST_RPC_ADDRESS={CASSANDRA_NODE_IPS[i]}",
            f"CASSANDRA_BROADCAST_ADDRESS={CASSANDRA_NODE_IPS[i]}",
            "LOCAL_JMX=no",  # Setting to 'no' allows remote JMX
        ],
        "volumes": [
            f"{os.path.join(DOCKER_MOUNTDIR, CASSANDRA_NODE_NAMES[i])}:{CASSANDRA_WORKDIR}",
            f"{os.path.join(DOCKER_MOUNTDIR, CASSANDRA_NODE_NAMES[i], "certs")}:/etc/cassandra/certs",
            f"{os.path.join(DOCKER_MOUNTDIR, CASSANDRA_NODE_NAMES[i], "cassandra.yaml")}:/etc/cassandra/cassandra.yaml",
            f"{os.path.join(DOCKER_MOUNTDIR, CASSANDRA_NODE_NAMES[i], "cassandra-rackdc.properties")}:/etc/cassandra/cassandra-rackdc.properties",
            f"{os.path.join(DOCKER_MOUNTDIR, "jmxremote.password")}:/etc/cassandra/jmxremote.password",
        ],
        "networks": ["cassandra-cluster"],
        # "hostname": f"{CASSANDRA_NODE_HOSTNAMES[i]}",
        "ports": [
            f"{CASSANDRA_NODE_RPC_PORTS[i]}:{CASSANDRA_NODE_RPC_PORTS[i]}",
            f"{CASSANDRA_NODE_GOSSIP_PORTS[i]}:{CASSANDRA_NODE_GOSSIP_PORTS[i]}",
            f"{CASSANDRA_NODE_SSL_GOSSIP_PORTS[i]}:{CASSANDRA_NODE_SSL_GOSSIP_PORTS[i]}",
            f"{CASSANDRA_NODE_JMX_PORTS[i]}:{CASSANDRA_NODE_JMX_PORTS[i]}",
        ],
        "extra_hosts": [
            f"{DOCKER_INTERNAL_HOST}:host-gateway",
        ],
        "dns": DOCKER_DNS,
        "deploy": {"resources": {"limits": {"cpus": node_cpus, "memory": node_memory}}},
        "healthcheck": {
            "test": [
                "CMD-SHELL",
                f"env JVM_OPTS= nodetool -u {CASSANDRA_INIT_USER} -pw {CASSANDRA_INIT_PASSWORD} status | grep UN || exit 1",
            ],
            "interval": "5s",
            "timeout": "10s",
            "retries": 24,
            "start_period": "20s",
        },
        "depends_on": {
            CASSANDRA_NODE_NAMES[j]: {"condition": "service_started"}
            for j in range(0, i)
        },
    }

cassandra_compose_yaml_path = os.path.join(
    LOCALHOST_WORKDIR, "cassandra-cluster.docker-compose.yml"
)
cassandra_compose_yaml_contents = yaml.dump(
    cassandra_compose_dict, default_flow_style=False, sort_keys=False, indent=4
)
with open(cassandra_compose_yaml_path, "w") as f:
    f.write(cassandra_compose_yaml_contents)

print(f"Successfully created: '{os.path.relpath(cassandra_compose_yaml_path)}'")
display(Markdown(f"```yaml\n{cassandra_compose_yaml_contents}\n```"))

Successfully created: 'cassandra-cluster.docker-compose.yml'


```yaml
name: cassandra-cluster
services:
    cassandra-node-1:
        image: cassandra:5.0
        container_name: cassandra-node-1
        environment:
        - CASSANDRA_USER=cassandra
        - MAX_HEAP_SIZE=2G
        - HEAP_NEWSIZE=1G
        - CASSANDRA_CLUSTER_NAME=cassandra-cluster.mavasbel.vpn.itam.mx
        - CASSANDRA_SEEDS=10.15.20.2:7001,10.15.20.2:7002,10.15.20.2:7003
        - CASSANDRA_LISTEN_ADDRESS=auto
        - CASSANDRA_RPC_ADDRESS=0.0.0.0
        - CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch
        - CASSANDRA_NATIVE_TRANSPORT_PORT=9041
        - CASSANDRA_STORAGE_PORT=7001
        - CASSANDRA_SSL_STORAGE_PORT=7501
        - JVM_OPTS=-Xms1G -Xmx2G -Dcassandra.native_transport_port=9041 -Dcassandra.storage_port=7001
            -Dcassandra.ssl_storage_port=7501 -Dcassandra.broadcast_rpc_address=cassandra-node-1.mavasbel.vpn.itam.mx
            -Dcassandra.broadcast_address=cassandra-node-1.mavasbel.vpn.itam.mx -Dcom.sun.management.jmxremote.authenticate=false
            -Dcom.sun.management.jmxremote.ssl=false -Dcassandra.jmx.remote.port=7201
        - CASSANDRA_BROADCAST_RPC_ADDRESS=10.15.20.2
        - CASSANDRA_BROADCAST_ADDRESS=10.15.20.2
        - LOCAL_JMX=no
        volumes:
        - .\mount\cassandra-node-1:/var/lib/cassandra
        - .\mount\cassandra-node-1\certs:/etc/cassandra/certs
        - .\mount\cassandra-node-1\cassandra.yaml:/etc/cassandra/cassandra.yaml
        - .\mount\cassandra-node-1\cassandra-rackdc.properties:/etc/cassandra/cassandra-rackdc.properties
        - .\mount\jmxremote.password:/etc/cassandra/jmxremote.password
        networks:
        - cassandra-cluster
        ports:
        - 9041:9041
        - 7001:7001
        - 7501:7501
        - 7201:7201
        extra_hosts:
        - host.docker.internal:host-gateway
        dns: &id001
        - 10.15.20.1
        deploy:
            resources:
                limits:
                    cpus: '2.0'
                    memory: 2G
        healthcheck:
            test:
            - CMD-SHELL
            - env JVM_OPTS= nodetool -u cassandra -pw cassandra status | grep UN ||
                exit 1
            interval: 5s
            timeout: 10s
            retries: 24
            start_period: 20s
        depends_on: {}
    cassandra-node-2:
        image: cassandra:5.0
        container_name: cassandra-node-2
        environment:
        - CASSANDRA_USER=cassandra
        - MAX_HEAP_SIZE=2G
        - HEAP_NEWSIZE=1G
        - CASSANDRA_CLUSTER_NAME=cassandra-cluster.mavasbel.vpn.itam.mx
        - CASSANDRA_SEEDS=10.15.20.2:7001,10.15.20.2:7002,10.15.20.2:7003
        - CASSANDRA_LISTEN_ADDRESS=auto
        - CASSANDRA_RPC_ADDRESS=0.0.0.0
        - CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch
        - CASSANDRA_NATIVE_TRANSPORT_PORT=9042
        - CASSANDRA_STORAGE_PORT=7002
        - CASSANDRA_SSL_STORAGE_PORT=7502
        - JVM_OPTS=-Xms1G -Xmx2G -Dcassandra.native_transport_port=9042 -Dcassandra.storage_port=7002
            -Dcassandra.ssl_storage_port=7502 -Dcassandra.broadcast_rpc_address=cassandra-node-2.mavasbel.vpn.itam.mx
            -Dcassandra.broadcast_address=cassandra-node-2.mavasbel.vpn.itam.mx -Dcom.sun.management.jmxremote.authenticate=false
            -Dcom.sun.management.jmxremote.ssl=false -Dcassandra.jmx.remote.port=7202
        - CASSANDRA_BROADCAST_RPC_ADDRESS=10.15.20.2
        - CASSANDRA_BROADCAST_ADDRESS=10.15.20.2
        - LOCAL_JMX=no
        volumes:
        - .\mount\cassandra-node-2:/var/lib/cassandra
        - .\mount\cassandra-node-2\certs:/etc/cassandra/certs
        - .\mount\cassandra-node-2\cassandra.yaml:/etc/cassandra/cassandra.yaml
        - .\mount\cassandra-node-2\cassandra-rackdc.properties:/etc/cassandra/cassandra-rackdc.properties
        - .\mount\jmxremote.password:/etc/cassandra/jmxremote.password
        networks:
        - cassandra-cluster
        ports:
        - 9042:9042
        - 7002:7002
        - 7502:7502
        - 7202:7202
        extra_hosts:
        - host.docker.internal:host-gateway
        dns: *id001
        deploy:
            resources:
                limits:
                    cpus: '2.0'
                    memory: 2G
        healthcheck:
            test:
            - CMD-SHELL
            - env JVM_OPTS= nodetool -u cassandra -pw cassandra status | grep UN ||
                exit 1
            interval: 5s
            timeout: 10s
            retries: 24
            start_period: 20s
        depends_on:
            cassandra-node-1:
                condition: service_started
    cassandra-node-3:
        image: cassandra:5.0
        container_name: cassandra-node-3
        environment:
        - CASSANDRA_USER=cassandra
        - MAX_HEAP_SIZE=2G
        - HEAP_NEWSIZE=1G
        - CASSANDRA_CLUSTER_NAME=cassandra-cluster.mavasbel.vpn.itam.mx
        - CASSANDRA_SEEDS=10.15.20.2:7001,10.15.20.2:7002,10.15.20.2:7003
        - CASSANDRA_LISTEN_ADDRESS=auto
        - CASSANDRA_RPC_ADDRESS=0.0.0.0
        - CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch
        - CASSANDRA_NATIVE_TRANSPORT_PORT=9043
        - CASSANDRA_STORAGE_PORT=7003
        - CASSANDRA_SSL_STORAGE_PORT=7503
        - JVM_OPTS=-Xms1G -Xmx2G -Dcassandra.native_transport_port=9043 -Dcassandra.storage_port=7003
            -Dcassandra.ssl_storage_port=7503 -Dcassandra.broadcast_rpc_address=cassandra-node-3.mavasbel.vpn.itam.mx
            -Dcassandra.broadcast_address=cassandra-node-3.mavasbel.vpn.itam.mx -Dcom.sun.management.jmxremote.authenticate=false
            -Dcom.sun.management.jmxremote.ssl=false -Dcassandra.jmx.remote.port=7203
        - CASSANDRA_BROADCAST_RPC_ADDRESS=10.15.20.2
        - CASSANDRA_BROADCAST_ADDRESS=10.15.20.2
        - LOCAL_JMX=no
        volumes:
        - .\mount\cassandra-node-3:/var/lib/cassandra
        - .\mount\cassandra-node-3\certs:/etc/cassandra/certs
        - .\mount\cassandra-node-3\cassandra.yaml:/etc/cassandra/cassandra.yaml
        - .\mount\cassandra-node-3\cassandra-rackdc.properties:/etc/cassandra/cassandra-rackdc.properties
        - .\mount\jmxremote.password:/etc/cassandra/jmxremote.password
        networks:
        - cassandra-cluster
        ports:
        - 9043:9043
        - 7003:7003
        - 7503:7503
        - 7203:7203
        extra_hosts:
        - host.docker.internal:host-gateway
        dns: *id001
        deploy:
            resources:
                limits:
                    cpus: '2.0'
                    memory: 2G
        healthcheck:
            test:
            - CMD-SHELL
            - env JVM_OPTS= nodetool -u cassandra -pw cassandra status | grep UN ||
                exit 1
            interval: 5s
            timeout: 10s
            retries: 24
            start_period: 20s
        depends_on:
            cassandra-node-1:
                condition: service_started
            cassandra-node-2:
                condition: service_started
networks:
    cassandra-cluster:
        driver: bridge

```

In [18]:
!docker compose -f cassandra-cluster.docker-compose.yml up -d --wait

 Network cassandra-cluster_cassandra-cluster  Creating
 Network cassandra-cluster_cassandra-cluster  Created
 Container cassandra-node-1  Creating
 Container cassandra-node-1  Created
 Container cassandra-node-2  Creating
 Container cassandra-node-2  Created
 Container cassandra-node-3  Creating
 Container cassandra-node-3  Created
 Container cassandra-node-1  Starting
 Container cassandra-node-1  Started
 Container cassandra-node-2  Starting
 Container cassandra-node-2  Started
 Container cassandra-node-3  Starting
 Container cassandra-node-3  Started
 Container cassandra-node-2  Waiting
 Container cassandra-node-3  Waiting
 Container cassandra-node-1  Waiting
 Container cassandra-node-1  Healthy
 Container cassandra-node-2  Healthy
 Container cassandra-node-3  Healthy
